In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores("global warming")

{'neg': 0.0, 'neu': 0.385, 'pos': 0.615, 'compound': 0.1531}

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from pandarallel import pandarallel
from tqdm.auto import tqdm
tqdm.pandas()
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
pandarallel.initialize(progress_bar=True, nb_workers=12, shm_size_mb=4000)

os.makedirs("classified", exist_ok=True)
analyzer = SentimentIntensityAnalyzer()
files = glob.glob("data/climate_tweets*.csv")
#print(files)
pd.set_option('display.max_colwidth', -1)

def classify(row):
    soup = BeautifulSoup(row.html, "lxml")
    s = []
    for child in soup.find("p").children:
        if child.name == None:
            s.append(child)
        elif child.name == "img":
            s.append(child["alt"])
        else:
            s.append(child.text)
    text_with_emoji = " ".join(s)
    row["text_with_emoji"] = text_with_emoji
    vs = analyzer.polarity_scores(text_with_emoji)
    for k,v in vs.items():
        row[k] = v
    return row

for f in tqdm(files):
    df = pd.read_csv(f, sep=";")
    df = df.parallel_apply(classify, axis=1)
    print("Complete - writing csv")
    new_filename = "classified/" + os.path.splitext(os.path.basename(f))[0] + ".csv"
    df.to_csv(new_filename, sep=";", index=False)

/home/ubuntu/.virtualenvs/twitter-analysis/lib/python3.7/site-packages/tqdm/_tqdm.py:634: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


New pandarallel memory created - Size: 4000 MB
Pandarallel will run on 12 workers


In [ ]:
%%time
# Classify NZ tweets
files = sorted(glob.glob("data/nz_tweets_*.csv"))
classify_tweets(files)

In [ ]:
%%time
# Classify climate change tweets
classify_tweets(sorted(glob.glob("/mnt/tweets/climate_tweets/histdaily-20190913T114130/combined.csv")))